In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import csv
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error
import sympy as sym
#import seaborn as sns

SOURCE_COVID_19 = "https://pomber.github.io/covid19/timeseries.json"
SOURCE_COUNTRIES_BY_POPULATION = "https://raw.githubusercontent.com/samayo/country-json/master/src/country-by-population.json"
SOURCE_GDP = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"
SOURCE_GDP_PER_CAPITA_JSON = "https://pkgstore.datahub.io/world-bank/ny.gdp.pcap.cd/data_json/data/5fa3074284c7b57091aa9bacc502ecb5/data_json.json"
SOURCE_GDP_PER_CAPITA_CSV = "https://datahub.io/world-bank/ny.gdp.pcap.cd/r/data.csv"
def get_data_json(source):
    return requests.get(source).json()

In [86]:
np.warnings.filterwarnings('ignore')

In [87]:
data = requests.get(SOURCE_COVID_19).json()

In [88]:
df = pd.DataFrame.from_dict(data)

In [89]:
def get_average_speed_of_propagation_list(data,status ='confirmed', days = len(data[list(data.keys())[0]]), reverse = False):
    min_days = days
    speed = {}
    for key in data:
        if(len(data[key]) < min_days):
            min_days = len(data[key])
    for key in data:
        speed[key] = round(data[key][min_days - 1][status] / min_days,3)
    speed = {key:value for key, value in sorted(speed.items(), key = lambda item:item[1], reverse = reverse)}
    return speed

In [90]:
def get_average_speed_of_propagation_list_non_zero_days(df,status ='confirmed', reverse = False , count = len(df.columns)):
    speed = {}
    for country_name in df.columns[:count]:
        country = pd.DataFrame.from_dict(data[country_name])
        country = country.loc[country[status] > 0 , ['date',status]].reset_index(drop = True)
        country.index = country.index + 1 # если что, убрать
        if country.empty:
            continue
        speed[country_name] = round(country.iloc[len(country)-1][status] / len(country) , 3)
    speed = {key:value for key, value in sorted(speed.items(), key = lambda item:item[1], reverse = reverse)}
    return speed

In [ ]:
get_average_speed_of_propagation_list_non_zero_days(df,status ='deaths', reverse = False)


In [92]:
def print_speed_stats(data,days = len(data[list(data.keys())[0]]), reverse = False):
    statuses = ['confirmed' , 'deaths' , 'recovered']
    for i in range(len(statuses)):
        print(f"Статистика за {days} дней:{statuses[i]}")
        speed = get_average_speed_of_propagation_list(data,status = statuses[i],days = days,reverse = reverse)
        print(f"Средняя скорость для {statuses[i]} к {days} дню (человек в день):")
        for key in speed:
            print(f"{key}:{speed[key]}")

In [ ]:
# Средняя скорость за все дни( с учётом нулевых)
print_speed_stats(data)

In [94]:
def print_speed_stats_non_zero_days(df, reverse = False , count = len(df.columns)):
    statuses = ['confirmed' , 'deaths' , 'recovered']
    for i in range(len(statuses)):
        print(f"Статистика за значимые( ненулевые) дни:{statuses[i]}")
        speed = get_average_speed_of_propagation_list_non_zero_days(df,statuses[i],reverse , count)
        print(f"Средняя скорость для {statuses[i]} (человек в день):")
        for key in speed:
            print(f"{key}:{speed[key]}")

In [ ]:
# Средняя скорость за все дни( без нулевых значений)
print_speed_stats_non_zero_days(df)

In [96]:
def draw_average_speed_of_propagation_list(data,days = len(data[list(data.keys())[0]]), reverse = False):   
    statuses = ['confirmed' , 'deaths' , 'recovered']
    colors = ['yellow' , 'red' ,'green']
    for i in range(3):
        fig , ax = plt.subplots()
        fig =plt.figure(figsize=(50,40))
        speed = get_average_speed_of_propagation_list(data,status = statuses[i],days = days,reverse = reverse)
        countries = list(speed.keys())
        values = list(speed.values())
        ax.bar(countries,values, color=colors[i])
        ax.set_title(f"{statuses[i]} for {days} days") 
        plt.show()
        

In [97]:
def draw_countries_covid_stats(df, count = len(df.columns)):
    for country_name in df.columns[:count]:
        country = df[country_name]
        x = [index for index in country.index]
        y = [obj['confirmed'] for obj in country]
        y1 = [obj['deaths'] for obj in country] 
        y2 = [obj['recovered'] for obj in country] 
        fig, ax = plt.subplots()
        ax.plot(x,y,label="confirmed")
        ax.plot(x,y1,label="deaths")
        ax.plot(x,y2,label="recovered")
        ax.set_title(country_name)  # Add a title to the axes.
        ax.set_xlabel("days")
        ax.set_ylabel("amount")
        ax.legend()

In [98]:
def create_country_stats_excel_file(df):
    writer = pd.ExcelWriter('covid-19.xlsx', engine='xlsxwriter')
    for country_name in df.columns:
        country = pd.DataFrame.from_dict(data[country_name])
        country_name = "".join([c for c in country_name if c.isalpha()])
        country.to_excel(writer ,sheet_name=country_name, index_label="day")
        #country.to_csv(f"{country_name}.csv" , index_label = "day")

In [99]:
def create_country_stats_excel_file_without_zeroes(df, by='confirmed'):
    writer = pd.ExcelWriter(f"covid-19_{by}.xlsx", engine='xlsxwriter')
    for country_name in df.columns:
        country = pd.DataFrame.from_dict(data[country_name])
        country = country.loc[country[by] > 0 , [by,'date']].reset_index(drop = True)
        country.index = country.index + 1 # если что, убрать
        #print(country)
        country_name = "".join([c for c in country_name if c.isalpha()])
        country.to_excel(writer ,sheet_name=country_name, index_label="day")
        #country.to_csv(f"{country_name}.csv" , index_label = "day")

        

In [100]:
create_country_stats_excel_file_without_zeroes(df)
create_country_stats_excel_file_without_zeroes(df, by='deaths')
create_country_stats_excel_file_without_zeroes(df, by= 'recovered')

In [101]:
def linear(x,a,b):
    return a*x + b

def exp(x,a,b,c):
    return a*np.exp(b*x)+c

def polynomial_square(x,a,b,c):
    return a*np.square(x) + b*x + c

def polynomial_cube(x,a,b,c,d):
    return a*np.power(x,3) + b*np.square(x) + c*x + d

def log(x,a,b,c):
    return a*np.log(b*x) + c



In [102]:
def get_curve_fit_with_params(x,y,method = "r2",maxfev = 1000):
    functions = {"linear":linear , "exp":exp , "polynomial_square":polynomial_square ,"polynomial_cube":polynomial_cube, "log":log}
    function =functions["linear"]
    name = "linear"
    params = [1,0]
    r_square = 0
    if method == "mse":
        mse = mean_squared_error(y, functions[name](x,*params))
    for key in functions:
        if key == "exp":
            opt_params , pcov = curve_fit(functions[key],x,y , p0 = (-1, 0.01, 1),maxfev = 5000)
        else:
            opt_params , pcov = curve_fit(functions[key],x,y,maxfev = 5000)
        if method == "r2":   
            r_square_cur = r2_score(y, functions[key](x,*opt_params))
            if r_square_cur > r_square:
                r_square = r_square_cur
                function = functions[key]
                name = key
                params = opt_params         
        else:
            mse_cur = mean_squared_error(y, functions[key](x,*opt_params))
            if mse_cur < mse:
                mse = mse_cur
                function = functions[key]
                name = key
                params = opt_params   
    if method == "mse":
        return (name, function , params, mse)    
    return (name, function , params, r_square)
        
        

In [103]:
def get_country_dependency(country, status, method):
    country_name = country['name']
    data = country['data']
    #print(country_name)
    #print(data)
    if data.empty:
        return None
    x = np.array(list(data.index))
    y = np.array(list(data[status]))
    # 
    if status =="deaths":
        threshold = 4
        if(len(x) < threshold) or (len(y) < threshold) or (len(y) != len(x)):
            return None
    name, func, params, res = get_curve_fit_with_params(x,y, method = method)
    mark = method 
    res = {"country": country_name , "criteria":status, "dependency":name ,"opt_params":params, method:res }
    #print(res)
    return res


In [104]:
def get_countries_by_status(df , status='confirmed' , count = len(df.columns) ):
    countries =[]
    for country_name in df.columns[:count]:
        country = pd.DataFrame.from_dict(data[country_name])
        country = country.loc[country[status] > 0 , ['date',status]].reset_index(drop = True)
        country.index = country.index + 1 # если что, убрать
        country.index
        countries.append({"name":country_name  ,"data":country})
    return countries

In [105]:
def get_countries_dependencies_by_status(df , status='confirmed' , count = len(df.columns) , method = "r2"):
    countries = get_countries_by_status(df,status,count)
    dependencies = []
    for country in countries:
        data = get_country_dependency(country,status, method)
        if data is not None:
            dependencies.append(data)
    return dependencies
 
    # {country:"Afghanistan", status:"confirmed", dependency:'exp' , function:"1,2*exp(2*x) + 4"}

In [106]:
def print_countries_dependencies_by_status(df , status='confirmed' , count = len(df.columns) , method = "r2"):
    countries = get_countries_dependencies_by_status(df,status , count , method)
    for country in countries:
        print(f"Страна: {country['country']} , критерий: {country['criteria']}, зависимость: {country['dependency']}")
        print(f"оптимальные параметры:" , end = " ")
        for param in country['opt_params']:
            print(param, end = ', ')
        print()
        print(f"{list(country.keys())[-1]}: {country[list(country.keys())[-1]]}")

In [ ]:
#Функции зависимости стран по количеству заражённых умерших и выздоровевших
# methods = ['r2' , 'mse']
#statuses ['confirmed' , 'deaths', 'recovered']
# Лучшие результаты при кубическом трёхчлене, при его отсутствии - при квадратном трёхчлене
print_countries_dependencies_by_status(df , status='confirmed' , count = len(df.columns) , method = "r2")

In [108]:
pop_data = requests.get(SOURCE_COUNTRIES_BY_POPULATION).json()

In [109]:
data["United States"] = data.pop("US")
df = df.rename(columns={"US": "United States"})

In [110]:
def get_procent_to_population(data,df,pop_data, count = len(df.columns)):
    result = []
    for country_name in df.columns[:count]: 
        country = pd.DataFrame.from_dict(data[country_name])    
        for state in pop_data:
            if state['country'] == country_name:
                population = state['population']
                break
        population = int(state['population'])
        confirmed = int(country.iloc[len(country)-1]['confirmed']) / population * 100
        dead = int(country.iloc[len(country)-1]['deaths']) / population * 100
        result.append({"country":country_name ,"population":population , "conf_to_pop":confirmed, "dead_to_pop":dead})
    return result

In [111]:
def print_procent_to_population(data,df,pop_data, count = len(df.columns)):
    countries = get_procent_to_population(data,df,pop_data,count) 
    for country in countries:
        print(f"Страна: {country['country']} , население: {country['population']} , процент заразившихся: {country['conf_to_pop']} , процент умерших: {country['dead_to_pop']}")
    

In [ ]:
# Кол-во заразившихся и умерших в процентах от всего населения
print_procent_to_population(data,df,pop_data)

In [121]:
#Проанализировать данные по странам по количеству зараженных и погибших в процентном отношении от всего населения в зависимости от валового внутреннего продукта страны и от среднедушевых доходов.

In [122]:
gdp_data_response = requests.get(SOURCE_GDP)
csv_path = "gdp.csv"

In [123]:
per_capita_data = requests.get(SOURCE_GDP_PER_CAPITA_JSON).json()

In [124]:
def set_csv_file(csv_path):
    csv_gdp_file = open(csv_path,'wb')
    csv_gdp_file.write(gdp_data_response.content)
    csv_gdp_file.close()
    

In [125]:
set_csv_file(csv_path)

In [126]:
def change(country):
    if country == "Bahamas, The":
        return "Bahamas"
    if country =="Brunei Darussalam":
        return "Brunei"
    elif country == "Congo, Dem. Rep.":
        return "Congo (Kinshasa)"
    elif country == "Congo, Rep.":
        return "Congo (Brazzaville)"
    elif country =="Egypt, Arab Rep.":
        return "Egypt"
    elif country == "Korea, Rep.":
        return "Korea, South"
    elif country == "Kyrgyz Republic":
        return "Kyrgyzstan"
    elif country =="Russian Federation":
        return "Russia"
    elif country == "St. Vincent and the Grenadines":
        return "Saint Vincent and the Grenadines"
    elif country == "Venezuela, RB":
        return "Venezuela"
    elif country == "Yemen, Rep.":
        return "Yemen"
    return country
    

In [127]:

def get_countries_gdp_csv(csv_path):
    countries = []
    length = 0
    with open(csv_path,"r") as csv_file:
        for line in csv_file:
            length +=1
    count = 0
    with open(csv_path,"r") as csv_file:
        line = csv_file.readline().split(",")
        while (line[0] != "Afghanistan"):
            line = csv_file.readline().split(",")
        while(count< length):
            line = csv_file.readline().split(",")
            if len(line) < 4:
                break
            if line[2] == "2016":
                countries.append({"country":change((line[0])) , "gdp":int(round(float(line[3])))})
            count+=1
    return countries

In [128]:
def get_countries_per_capita(per_capita_data):
    countries = []
    length = len(per_capita_data)
    count = 0
    _iter = iter(per_capita_data)
    key = next(_iter)

    while key['Country Name']!= 'Afghanistan':
        key = next(_iter)
        count+=1
    while(count + 1 < length):
        key = next(_iter)
        count+=1
        if(int(key['Year']) == 2016):
            countries.append({"country":change((key['Country Name'])) , "per_capita":int(key['Value'])})
    return countries

In [ ]:
get_countries_per_capita(per_capita_data)

In [130]:
# ВВП в миллиардах долларов
# Среднедушевой доход в тысячах долларов

In [145]:
def get_country_list_by_param(percentage_list,param_list,param = "gdp", divider = 10**9):
    _list = []
    for country in percentage_list:
        for state in param_list:
            if country['country'] == state['country']:
                country[param] = float(state[param]) / divider
                _list.append(country)
                break
    return _list

In [163]:
gdp_csv_list = get_countries_gdp_csv(csv_path)
per_capita_list = get_countries_per_capita(per_capita_data)
states_percentage = get_procent_to_population(data,df,pop_data) 
countries_with_gdp = get_country_list_by_param(states_percentage,gdp_csv_list)
countries_with_gdp_and_capita = get_country_list_by_param(countries_with_gdp,per_capita_list, param="per_capita", divider = 10**3)

In [135]:
#statuses('conf_to_pop' , "dead_to_pop")

In [156]:
def get_data_by_param_and_status(countries, param = "gdp", status='conf_to_pop'):
    _sorted = sorted(countries, key=lambda k: k[param]) 
    _x = []
    _y = []
    for country in _sorted:
        _x.append(country[param])
        _y.append(country[status])
    #return np.array(_x) , np.array(_x)
    return _x , _y

In [169]:
def get_dependency_by_param_and_status(countries, param="gdp", status='conf_to_pop', method = "r2"):
    x,y = get_data_by_param_and_status(countries,param,status)
    name, func , params , res = get_curve_fit_with_params(np.array(x),np.array(y),method)
    return {"criteria":status,"param":param, "dependency":name ,"opt_params":params, method:res }

In [175]:
def print_dependency_by_param_and_status(countries, param = "gdp", status='conf_to_pop', method = "r2"):
    res = get_dependency_by_param_and_status(countries, param, status, method)
    _param = "ВВП" if res['param']=="gdp" else "Среднешушевой доход"
    _param_con = "ВВП" if res['param']=="gdp" else "Среднешушевого дохода"
    print(f"Кривая для {_param_con}:")
    criteria = f"Процент заражённых/{_param}" if res['criteria'] =="conf_to_pop" else f"Процент умерших/{_param}"
    print(f"Критерий: {criteria}, зависимость: {res['dependency']}")
    print("Оптимальные параметры:" , end = " ")
    for param in res['opt_params']:
        print(param, end = ", ")

In [176]:
#statuses('conf_to_pop' , "dead_to_pop")
#params('gdp','per_capita')

In [181]:
print_dependency_by_param_and_status(countries_with_gdp_and_capita,param="gdp", status='conf_to_pop', method = "r2")

Кривая для ВВП:
Критерий: Процент заражённых/ВВП, зависимость: polynomial_cube
Оптимальные параметры: 1.2103010680067706e-12, -3.056313949914963e-08, 0.00017061935522433622, 0.08954109339445318, 